In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
content = """
LinkedIn: linkedin.com/in/kexin-wei-611s96

Experience:

Research and Development Engineer (Jun 2022 — Now)
Creative Medtech Solutions Pte.Ltd., Singapore


Engineered robotics control systems with 4-DoF robot kinematics optimization
Implemented AI-based medical image processing (segmentation, DICOM, US/MRI registration)
Enhanced system security and developed GUI
Integrated electromagnetic sensors for HIFU treatment visualization
Developed power calibration and motion tracking algorithms
Filed 3 medical algorithm patents


Research Assistant - Internship (Jul 2018 — Sep 2018)
Beijing Surgerii Technology Co., Ltd. and Kii Lab of SJTU


Evaluated 3D positioning for multi-backbone robot
Analyzed needle insertion experiments and improved mechanical design

Leadership:
Scrum Master (Mar 2024 — Now)
Creative Medtech Solutions Pte.Ltd., Singapore

Led Git workflow standardization
Integrated GoogleTest framework
Implemented Scrum processes and code review standards

Education:

National University of Singapore (Aug 2020 — May 2022)


Master of Engineering in Medical Robotics
GPA: 4.63/5.00
Thesis: Deep Reinforcement Learning for Robot-assisted Surgical Training
Teaching Assistant for Motor Characteristics and Machine Vision modules


Tongji University, China (Sep 2014 — Jul 2019)


Bachelor of Engineering in Mechanical Design, Manufacture and Automation
GPA: 4.61/5.00

Skills:

Languages: English (IELTS 7, GRE 325), German (TestDaf 16/20, DSH 3/3), Mandarin (native)
Programming: C++, Python, Docker, ROS, Pytorch, Qt, Matlab, Galil, VTK
CAD/CAE: Solidworks, UG, AutoCAD

Awards:

2023: Hackathon "Runner Up" Prize at bio LLMs (4Catalyzer)
2018: 2nd Prize in Mathematics Modeling (CUMCM), Honorable Mention (MCM/ICM)
2016: Group 1st Prize in "Shangtu Cup" (AutoCAD Contest)
Multiple scholarship awards
""".strip()

In [3]:
job_description = """
MiRXES is a Singapore-headquartered molecular diagnostic company with R&D, manufacturing, and clinical lab operations in Singapore, USA, Japan, and China. We specialize in microRNA technologies and the development, manufacturing, and provision of diagnostic test kits and clinical services. Our mission is to enable early disease detection and provide insights for better patient care by harnessing the power of microRNA to augment traditional testing. Our vision is to be the world’s leading developer and provider of accurate, actionable, and affordable microRNA-powered diagnostic tests.

Job Location: Neuros

Job Summary

The Genomics lab is looking for a Research Scientist, whose main focus is to develop NGS workflow for academic and clinical research service, and train technical personnel, leading to the implementation of these new workflow in a commercial service lab environment. The appointment holder is expected to have advanced degree with working experience in NGS technologies. The ideal candidate would be well-balanced between having the discipline to work in a commercial service laboratory environment, and a passion for translating innovative, newly developed laboratory tests for commercial implementation.

Roles and Responsibilities
• Lead the molecular assay development and validation efforts in a commercial service lab setting.
• Demonstrate hand-on leadership to successfully implement NGS service workflow in the commercial service laboratory.
• Design, plan and execute validation tests, establish QC criteria, write validation plan and report.
• Lead the SOP writing and technology transfer to the operations team.
• Propose and implement process improvements and EQA.
• Work closely with other Scientists and Product Managers to provide scientific inputs related to company products and testing services.
• Support other commercial activities as required.

Desired Skills & Competencies
• PhD. in related Science background with 1 – 2 years’ experience in molecular biology or related field.
• MSc. in related Science background with 3 – 5 years’ experience in molecular biology or related field.
• Demonstrated independent research & development work in a molecular biology lab setting.
• Strong understanding and working experiences on NGS technologies is highly desirable.
• Experience in developing and implementing new molecular assays in a commercial service laboratory is an added advantage.
• Experience in single cell sequencing will be added advantage.
• Be a ‘hands-on’ person who welcomes challenges, works independently or as part of a team, and can communicate effectively at all levels.

We appreciate your interest in the above-mentioned position, however, due to overwhelming response, only shortlisted candidates will be contacted.
""".strip()

# Simple Version

In [4]:
import google.generativeai as genai
from dotenv import load_dotenv
import os
import json
import re

# Load environment variables from .env file
load_dotenv()

# Access the API key from the environment variable
api_key = os.getenv("SECRET_KEY")

# Configure the API key for the genai library
genai.configure(api_key=api_key)

In [5]:
class CVAgent:
    def __init__(self):
        self.model_editor = genai.GenerativeModel(
            "gemini-1.5-flash",
            generation_config=genai.GenerationConfig(temperature=0.1),
        )

        self.model_creator = genai.GenerativeModel(
            "gemini-1.5-flash",
            generation_config=genai.GenerationConfig(temperature=0.1),
        )

    def _call_creator(self, content, job_description):
        creator_prompt = (
            f"You are a editor that draft a cover letter within 700 words"
            f"based on the content:{content}"
            f"given the job description: {job_description}"
        )
        response = self.model_creator.generate_content(creator_prompt)
        return response.text

    def _call_editor(self, content):
        editor_prompt = f"You will correct grammar, and make style consistent. Here is the content: {content}"
        response = self.model_editor.generate_content(editor_prompt)
        return response.text

    def __call__(self, content, job_description):
        draft = self._call_creator(content, job_description)
        edited_draft = self._call_editor(draft)
        return edited_draft

In [6]:
agent = CVAgent()
result = agent(content=content, job_description=job_description)

In [7]:
print(result)

[Your Name]
[Your Address]
[Your Phone Number]
[Your Email Address]

[Date]

Hiring Manager
MiRXES
[Address if known, otherwise omit]

Dear Hiring Manager,

I am writing to express my strong interest in the Research Scientist position at MiRXES, as advertised on [Platform where you saw the advertisement]. My background in medical robotics and AI, coupled with my proven ability to translate research into practical commercial applications, aligns well with the job description's requirements. While my experience is not directly in next-generation sequencing (NGS), my transferable skills and rapid learning aptitude make me a compelling candidate.

My experience as a Research and Development Engineer at Creative Medtech Solutions Pte. Ltd. provided a robust foundation in developing and implementing complex systems.  I successfully engineered robotics control systems, implemented AI-based medical image processing techniques (including segmentation, DICOM, and US/MRI registration), and develo

# With Supervisor

In [8]:
import getpass
import os
import json

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import ChatOpenAI
from langchain_core.messages import (
    AnyMessage,
    SystemMessage,
    HumanMessage,
    ToolMessage,
    AIMessage,
)

In [9]:
from langchain_core.tools import tool


@tool
def ask_sub_agent(prompt: str) -> str:
    """Useful for when you need to create a sub-agent to perform a specific task.
    The input to this tool should be a clear and specific description of the task the sub-agent should perform.
    """
    model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
    response = model.invoke(prompt)
    return response.content


tools = [ask_sub_agent]

In [10]:
system_prompt = """
You are a supervisor agent.
Your job is to manage a team of sub-agents with minimal capacity of 2 and maximal 5.

You should create a plan according to the query first,
and assign each task to a sub-agent and collect their result.

You will need to check the result of each sub-agent and make sure they are correct.
And the last step is to make sure the final answer match the query and return result back.
""".strip()

In [11]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)

supervisor = create_react_agent(
    model=llm,
    tools=tools,
    checkpointer=MemorySaver(),
    state_modifier=system_prompt,
)


def ask_agent(content):
    input = {
        "messages": [HumanMessage(content=content)],
    }
    for s in supervisor.stream(
        input,
        stream_mode="values",
        config={
            "configurable": {"thread_id": "thread-1"},
        },
    ):
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [12]:
question = (
    f"can you help me to write a CV give the content and job description? "
    f"content:{content}\n job description:{job_description}"
)
ask_agent(question)

================================ Human Message =================================

can you help me to write a CV give the content and job description? content:LinkedIn: linkedin.com/in/kexin-wei-611s96

Experience:

Research and Development Engineer (Jun 2022 — Now)
Creative Medtech Solutions Pte.Ltd., Singapore


Engineered robotics control systems with 4-DoF robot kinematics optimization
Implemented AI-based medical image processing (segmentation, DICOM, US/MRI registration)
Enhanced system security and developed GUI
Integrated electromagnetic sensors for HIFU treatment visualization
Developed power calibration and motion tracking algorithms
Filed 3 medical algorithm patents


Research Assistant - Internship (Jul 2018 — Sep 2018)
Beijing Surgerii Technology Co., Ltd. and Kii Lab of SJTU


Evaluated 3D positioning for multi-backbone robot
Analyzed needle insertion experiments and improved mechanical design

Leadership:
Scrum Master (Mar 2024 — Now)
Creative Medtech Solutions Pte.Ltd., 